# **Araç Satış Tahminleme**

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


In [12]:
# Dosyayı yükle
data = pd.read_csv('ikinciyeni_veriseti.csv')

In [13]:
data.head()

,Ekleme tarihi,Açılış fiyatı,Kapanış fiyatı,Kaput,SoLÖnÇamurluk,SoLÖnKapı,SoLArkaKapı,SoLArkaÇamurluk,bagaj,SaĞÖnÇamurluk,...,Alarm,4 X 4,Park Asistanı,CD Değiştiricili Radyo,Vakumlu Kapı,3 Araç Anahtarı Mevcut (Toplam),Kapı Hava Yastığı,Radyo - Kasetçalar,Merkezi Kilit,Döşeme: Suni Deri
0,2018-02-10,63.0,TEKLİF YOK,-,-,B,D,B,-,-,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
1,2018-02-10,45.0,62.900,B,B,B,-,B,-,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
2,2018-02-10,79.0,95.800,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
3,2018-02-09,49.0,74.500,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
4,2018-02-09,97.0,TEKLİF YOK,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK


In [14]:
data.shape

(379, 87)

In [15]:
# Kapanış fiyatı sütununu temizleme
data['Kapanış fiyatı'] = data['Kapanış fiyatı'].str.replace('TEKLİF YOK', '0').astype(float)

# Kapanış fiyatı sütunundaki değeri 0 olan satırları silme
data = data[data['Kapanış fiyatı'] != 0]

# Gerekli olmayan sütunları çıkarma (tarih ve metin sütunları)
columns_to_drop = ['Ekleme tarihi', 'İlk Tescil Tarihi', 'Son Muayene Tarihi','Döşeme: Suni Deri','Merkezi Kilit','Radyo - Kasetçalar','Kapı Hava Yastığı','Vakumlu Kapı','Park Asistanı','Ok Gösterimli Navigasyon Cihazı'] 
data_cleaned = data.drop(columns=columns_to_drop) #data_cleaned: Sütunları çıkarılmış olan yeni veri çerçevesi.

# Aynı veriyi içeren sütunları tespit etme
nunique = data_cleaned.apply(pd.Series.nunique) #data_cleaned.apply(pd.Series.nunique): Her bir sütundaki benzersiz değerlerin sayısını hesaplar.
cols_to_drop = nunique[nunique == 1].index #nunique[nunique == 1].index: Tüm değerleri aynı olan (benzersiz değer sayısı 1 olan) sütunların isimlerini seçer.

# Bu sütunları çıkarma
data_cleaned = data_cleaned.drop(columns=cols_to_drop)

# "-" değerlerini NaN yapma
data_cleaned = data_cleaned.replace('-', np.nan)

# NaN değerlerini rastgele "B" veya "D" ile doldurma
data_cleaned = data_cleaned.apply(lambda x: x.fillna(np.random.choice(['B', 'D']) if x.dtype == 'object' else x)) #.apply(lambda x: ...): Her bir sütunu ayrı ayrı işleyen bir lambda fonksiyonu uygular.
#x.fillna(...): Her sütundaki eksik değerleri (NaN) belirli bir değerle doldurur.
#np.random.choice(['B', 'D']) if x.dtype == 'object' else x: Sütun tipi object ise, eksik değerleri rastgele olarak 'Y' veya 'N' ile doldurur; değilse doldurma işlemi yapmaz.


In [16]:
data_cleaned.sample(10)

,Açılış fiyatı,Kapanış fiyatı,Kaput,SoLÖnÇamurluk,SoLÖnKapı,SoLArkaKapı,SoLArkaÇamurluk,bagaj,SaĞÖnÇamurluk,SaĞÖnKapı,...,Döşeme: Kumaş/Deri Kombinasyonu,Xenon Far,Elektrik Ayarlı Koltuk,Diz Hava Yastığı,Döşeme: Deri,Hafızalı Koltuk,CD Çalarlı Radyo,Döşeme: Alcantara/Deri,4 X 4,CD Değiştiricili Radyo
135,75.0,102.1,B,D,D,B,B,D,B,B,...,YOK,YOK,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK
205,79.0,103.1,B,B,D,B,B,D,B,B,...,YOK,VAR,VAR,YOK,YOK,VAR,YOK,VAR,YOK,YOK
304,99.0,99.1,B,D,D,B,B,D,B,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
308,85.0,109.3,B,D,D,B,B,D,B,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
16,90.0,90.2,B,B,D,B,B,D,B,B,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
218,29.0,41.1,B,D,D,B,B,D,B,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
225,45.0,70.8,B,D,B,B,B,D,B,B,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
82,83.0,83.1,B,D,D,B,B,D,B,B,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
7,29.9,48.5,D,D,D,B,B,D,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
226,79.0,79.1,B,D,D,B,D,B,B,B,...,YOK,YOK,VAR,YOK,YOK,VAR,YOK,VAR,YOK,YOK


In [17]:
data.isnull().sum() # Missing data kontrolü

Ekleme tarihi                      0
Açılış fiyatı                      0
Kapanış fiyatı                     0
Kaput                              0
SoLÖnÇamurluk                      0
                                  ..
3 Araç Anahtarı Mevcut (Toplam)    0
Kapı Hava Yastığı                  0
Radyo - Kasetçalar                 0
Merkezi Kilit                      0
Döşeme: Suni Deri                  0
Length: 87, dtype: int64

In [18]:
# Özellik ve hedef değişkenleri ayırma
X = data_cleaned.drop(columns=['Kapanış fiyatı'])
y = data_cleaned['Kapanış fiyatı']

# Kategorik sütunları one-hot encoding ile dönüştürme
categorical_features = X.select_dtypes(include=['object']).columns.tolist() #categorical_features: X veri çerçevesindeki kategorik sütunların isimlerini içeren liste.
numerical_features = X.select_dtypes(include=['number']).columns.tolist() #numerical_features: X veri çerçevesindeki sayısal sütunların isimlerini içeren liste.

# ColumnTransformer ile sayısal ve kategorik sütunları dönüştürme
preprocessor = ColumnTransformer( #ColumnTransformer: Farklı sütunlar için farklı dönüştürme işlemlerini uygulamaya yarayan bir sklearn aracı.
    transformers=[
        ('num', StandardScaler(), numerical_features), #'num', StandardScaler(), numerical_features: Sayısal sütunları standartlaştırmak için StandardScaler kullanılır.
        ('cat', OneHotEncoder(), categorical_features) #'cat', OneHotEncoder(), categorical_features: Kategorik sütunları one-hot encoding ile dönüştürmek için OneHotEncoder kullanılır.
    ])

# Pipeline ile dönüşüm işlemlerini gerçekleştirme
pipeline = Pipeline(steps=[ #Pipeline: Veriyi işlemden geçirmek için adımların sırayla uygulanmasını sağlayan bir sklearn aracı.
    ('preprocessor', preprocessor) #steps=[('preprocessor', preprocessor)]: Ön işlemciyi (preprocessor) pipeline'a ekler.
])

# Verileri dönüştürme ve eğitim-test setlerine ayırma
X_processed = pipeline.fit_transform(X) #pipeline.fit_transform(X): X verisini pipeline içindeki işlemlerden geçirerek dönüştürür.
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42) #train_test_split(X_processed, y, test_size=0.2, random_state=42): Dönüştürülmüş veriyi eğitim ve test setlerine böler.
#X_train, X_test: Özellikler için eğitim ve test setleri.
#y_train, y_test: Hedef değişken için eğitim ve test setleri.
#test_size=0.2: Verinin %20'si test setine, %80'i eğitim setine ayrılır.
#random_state=42: Rastgelelik için sabit bir başlangıç değeri, böylece sonuçlar tekrarlanabilir.

In [19]:
data.dtypes

Ekleme tarihi                       object
Açılış fiyatı                      float64
Kapanış fiyatı                     float64
Kaput                               object
SoLÖnÇamurluk                       object
                                    ...   
3 Araç Anahtarı Mevcut (Toplam)     object
Kapı Hava Yastığı                   object
Radyo - Kasetçalar                  object
Merkezi Kilit                       object
Döşeme: Suni Deri                   object
Length: 87, dtype: object

In [20]:
# MLP modelini oluşturma
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42) #MLPRegressor: sklearn kütüphanesindeki bir çok katmanlı perceptron regresyon modelidir.
#hidden_layer_sizes=(100, 50): Gizli katmanların boyutlarını belirtir. Bu durumda, iki gizli katman vardır: ilki 100 nöronlu, ikincisi 50 nöronlu.
#max_iter=1000: Maksimum 1000 iterasyon boyunca modeli eğitir.
#random_state=42: Rastgelelik için sabit bir başlangıç değeri, böylece sonuçlar tekrarlanabilir.

# Model eğitimi
mlp.fit(X_train, y_train) #mlp.fit(X_train, y_train): MLP modelini eğitim verileri (X_train) ve hedef değişken (y_train) kullanarak eğitir.

# Tahmin yapma
y_pred_train = mlp.predict(X_train) #mlp.predict(X_train): Eğitim seti üzerinde tahminler yapar ve tahmin edilen değerleri y_pred_train değişkenine atar.
y_pred_test = mlp.predict(X_test) #mlp.predict(X_test): Test seti üzerinde tahminler yapar ve tahmin edilen değerleri y_pred_test değişkenine atar.

# Eğitim ve test hatalarını hesaplama
mse_train = mean_squared_error(y_train, y_pred_train) #mean_squared_error(y_train, y_pred_train): Eğitim seti üzerindeki ortalama kare hatayı (MSE) hesaplar.
mse_test = mean_squared_error(y_test, y_pred_test) #mean_squared_error(y_test, y_pred_test): Test seti üzerindeki ortalama kare hatayı (MSE) hesaplar.

print("Eğitim MSE:", mse_train)
print("Test MSE:", mse_test)

Eğitim MSE: 338.0238099839451
Test MSE: 281.9929644491422
